# **Superstore Sales Analysis**

## **Project Summary**
**Goal**

A global superstore that ships internationally has provided a dataset of orders from 2014-2017, and has tasked me to create a dashboard for upper management to view key metrics on sales and understand latest trends in the business.

**Overview**

In this notebook, I conduct initial data exploration and clean the dataset provided before exporting it for the creation of the dashboard and other visualisations in Tableau.

**Steps**
1. Data Source and Setup
2. Assess and Clean Data
3. Analyse Trends (Tableau)
4. Build Dashboard (Tableau)

## **Step 1: Data Source and Setup**

**Data Source**

This sample dataset was taken from Kaggle (https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting)

**Setup**

In [14]:
# Import operational libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [33]:
# Read in data
orders = pd.read_csv('global_superstore_2018_orders.csv')
returns = pd.read_csv('global_superstore_2018_returns.csv')

## **Step 2: Assess and Clean Data**

### **Orders dataset:**

In [44]:
# View example rows
orders.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,State,Country,Region,Market,Product ID,Product Name,Sub-Category,Category,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,24599,IN-2017-CA120551-42816,22/3/2017,29/3/2017,Standard Class,CA-120551,Cathy Armstrong,Home Office,NaN,Herat,Hirat,Afghanistan,Southern Asia,Asia Pacific,FUR-BO-4861,"Ikea Library with Doors, Mobile",Bookcases,Furniture,731.82,2,0.0,102.42,39.66,Medium
1,29465,ID-2015-BD116051-42248,1/9/2015,4/9/2015,Second Class,BD-116051,Brian Dahlen,Consumer,NaN,Herat,Hirat,Afghanistan,Southern Asia,Asia Pacific,OFF-SU-2988,"Acme Scissors, Easy Grip",Supplies,Office Supplies,243.54,9,0.0,104.49,18.72,Medium
2,24598,IN-2017-CA120551-42816,22/3/2017,29/3/2017,Standard Class,CA-120551,Cathy Armstrong,Home Office,NaN,Herat,Hirat,Afghanistan,Southern Asia,Asia Pacific,TEC-MA-4211,"Epson Receipt Printer, White",Machines,Technology,346.32,3,0.0,13.77,14.10,Medium
3,24597,IN-2017-CA120551-42816,22/3/2017,29/3/2017,Standard Class,CA-120551,Cathy Armstrong,Home Office,NaN,Herat,Hirat,Afghanistan,Southern Asia,Asia Pacific,FUR-FU-5726,"Rubbermaid Door Stop, Erganomic",Furnishings,Furniture,169.68,4,0.0,79.68,11.01,Medium
4,29464,ID-2015-BD116051-42248,1/9/2015,4/9/2015,Second Class,BD-116051,Brian Dahlen,Consumer,NaN,Herat,Hirat,Afghanistan,Southern Asia,Asia Pacific,OFF-EN-3664,"Cameo Interoffice Envelope, with clear poly wi...",Envelopes,Office Supplies,203.88,4,0.0,24.36,5.72,Medium


In [59]:
# View columns, data types, and null values
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          51290 non-null  int64         
 1   Order ID        51290 non-null  object        
 2   Order Date      51290 non-null  datetime64[ns]
 3   Ship Date       51290 non-null  object        
 4   Ship Mode       51290 non-null  object        
 5   Customer ID     51290 non-null  object        
 6   Customer Name   51290 non-null  object        
 7   Segment         51290 non-null  object        
 8   Postal Code     9994 non-null   float64       
 9   City            51290 non-null  object        
 10  State           51290 non-null  object        
 11  Country         51290 non-null  object        
 12  Region          51290 non-null  object        
 13  Market          51290 non-null  object        
 14  Product ID      51290 non-null  object        
 15  Pr

We see that the only column with null values are postal codes. For our analysis, this should not be an issue

For this dataset, the only column that needs reformatting and will affect our analysis is the 'order date'.

In [64]:
# Changing order date to datetime format
orders['Order Date'] = pd.to_datetime(orders['Order Date'], format = '%d/%m/%Y')

0    2017
1    2015
2    2017
3    2017
4    2015
Name: Order Date, dtype: int32


In [20]:
# Check for duplicate rows
orders.duplicated().sum()

np.int64(0)

In [22]:
# Check for outliers in numerical columns
orders.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit,Shipping Cost
count,51290.00000,9994.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,25645.50000,55190.379428,246.490581,3.476545,0.142908,28.610982,26.478567
std,14806.29199,32063.693350,487.565361,2.278766,0.212280,174.340972,57.251373
min,1.00000,1040.000000,0.444000,1.000000,0.000000,-6599.978000,1.002000
25%,12823.25000,23223.000000,30.758625,2.000000,0.000000,0.000000,2.610000
50%,25645.50000,56430.500000,85.053000,3.000000,0.000000,9.240000,7.790000
75%,38467.75000,90008.000000,251.053200,5.000000,0.200000,36.810000,24.450000
max,51290.00000,99301.000000,22638.480000,14.000000,0.850000,8399.976000,933.570000


There appears to be some outliers in profit and shipping cost. We can explore this further.

In [29]:
def find_outliers(data, column):
    lq = data[column].quantile(0.25)
    uq = data[column].quantile(0.75)
    iqr= uq - lq
    outliers = data[(data[column] < lq - 1.5 * iqr)|(data[column] > uq + 1.5 * iqr)]
    return outliers

find_outliers(orders, 'Profit').sort_values(by = 'Profit', ascending = True).head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,State,Country,Region,Market,Product ID,Product Name,Sub-Category,Category,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
47386,39069,CA-2016-CS12505140-42700,26/11/2016,3/12/2016,Standard Class,CS-125051406,Cindy Stewart,Consumer,43130.0,Lancaster,Ohio,United States,Eastern US,USCA,TEC-MA-3854,Cubify CubeX 3D Printer Double Head Print,Machines,Technology,4499.985,5,0.7,-6599.9780,451.63,Low
38044,48663,TU-2016-DM3345134-42639,26/9/2016,26/9/2016,Same Day,DM-3345134,Denise Monton,Corporate,NaN,Bornova,Izmir,Turkey,Western Asia,Asia Pacific,TEC-PH-5267,"Motorola Smart Phone, Cordless",Phones,Technology,3085.344,12,0.6,-4088.3760,51.89,Critical
46865,31980,US-2017-GT14635140-43044,5/11/2017,5/11/2017,Same Day,GT-146351408,Grant Thornton,Corporate,27217.0,Burlington,North Carolina,United States,Southern US,USCA,TEC-MA-3855,Cubify CubeX 3D Printer Triple Head Print,Machines,Technology,7999.980,4,0.5,-3839.9904,674.82,High
49593,41071,CA-2014-LF17185140-41845,25/7/2014,29/7/2014,Standard Class,LF-171851402,Luke Foster,Consumer,78207.0,San Antonio,Texas,United States,Central US,USCA,OFF-BI-4340,GBC DocuBind P400 Electric Binding System,Binders,Office Supplies,2177.584,8,0.8,-3701.8928,172.07,Medium
43040,34308,CA-2017-SR20425140-42843,18/4/2017,24/4/2017,Standard Class,SR-204251404,Sharelle Roach,Home Office,80027.0,Louisville,Colorado,United States,Western US,USCA,TEC-MA-5082,Lexmark MX611dhe Monochrome Laser Printer,Machines,Technology,2549.985,5,0.7,-3399.9800,120.31,Medium
33287,10458,IT-2016-SS20140104-42531,10/6/2016,14/6/2016,Standard Class,SS-20140104,Saphhira Shifley,Corporate,NaN,Lisbon,Lisboa,Portugal,Southern Europe,Europe,OFF-AP-4745,"Hoover Stove, White",Appliances,Office Supplies,3399.660,12,0.5,-3059.8200,286.87,Medium
31563,29513,ID-2016-SN2056097-42627,14/9/2016,14/9/2016,Same Day,SN-2056097,Skye Norling,Home Office,NaN,Lahore,Punjab,Pakistan,Southern Asia,Asia Pacific,TEC-PH-3148,"Apple Smart Phone, Full Size",Phones,Technology,3499.155,11,0.5,-3009.4350,333.10,Medium
44030,36288,US-2017-HG14965140-43077,8/12/2017,14/12/2017,Standard Class,HG-149651402,Henry Goldwyn,Corporate,60653.0,Chicago,Illinois,United States,Central US,USCA,OFF-BI-4821,Ibico EPK-21 Electric Binding System,Binders,Office Supplies,1889.990,5,0.8,-2929.4845,129.51,Medium
25043,46219,LH-2017-JC610575-42763,28/1/2017,30/1/2017,Second Class,JC-610575,Julie Creighton,Corporate,NaN,Vilnius,Vilnius,Lithuania,Northern Europe,Europe,FUR-TA-3345,"Barricks Conference Table, Rectangular",Tables,Furniture,2171.160,8,0.7,-2750.2800,390.09,High
47482,34448,CA-2015-NF18385140-42353,15/12/2015,18/12/2015,First Class,NF-183851406,Natalie Fritzler,Consumer,43055.0,Newark,Ohio,United States,Eastern US,USCA,TEC-MA-3854,Cubify CubeX 3D Printer Double Head Print,Machines,Technology,1799.994,2,0.7,-2639.9912,219.62,High


It appears that the products which incurred the greatest losses were either machines or binders sold at high discount rates over 50%.

It is worth exploring this further and consider stopping the sales of these specific products altogether.

However, as these datapoints are plausible, I will not drop them.

In [31]:
find_outliers(orders, 'Shipping Cost').sort_values(by = 'Shipping Cost', ascending = False).head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,State,Country,Region,Market,Product ID,Product Name,Sub-Category,Category,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
46569,32298,CA-2015-RH19495140-42216,31/7/2015,31/7/2015,Same Day,RH-194951406,Rick Hansen,Consumer,10024.0,New York City,New York,United States,Eastern US,USCA,TEC-AC-5605,Plantronics CS510 - Over-the-Head monaural Wir...,Accessories,Technology,2309.650,7,0.0,762.1845,933.57,Critical
1594,26341,IN-2016-JR162107-42405,5/2/2016,7/2/2016,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,"Novimex Executive Leather Armchair, Black",Chairs,Furniture,3709.395,9,0.1,-288.7650,923.63,Critical
1761,25330,IN-2016-CR127307-42660,17/10/2016,18/10/2016,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,"Nokia Smart Phone, with Caller ID",Phones,Technology,5175.171,9,0.1,919.9710,915.49,Medium
15856,13524,ES-2016-KM1637548-42397,28/1/2016,30/1/2016,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,Germany,Western Europe,Europe,TEC-PH-5267,"Motorola Smart Phone, Cordless",Phones,Technology,2892.510,5,0.1,-96.5400,910.16,Medium
34309,47221,SG-2016-RH9495111-42679,5/11/2016,6/11/2016,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,Senegal,Western Africa,Africa,TEC-CO-6011,"Sharp Wireless Fax, High-Speed",Copiers,Technology,2832.960,8,0.0,311.5200,903.04,Critical
1273,22732,IN-2016-JM156557-42549,28/6/2016,1/7/2016,Second Class,JM-156557,Jim Mitchum,Corporate,NaN,Sydney,New South Wales,Australia,Oceania,Asia Pacific,TEC-PH-5842,"Samsung Smart Phone, with Caller ID",Phones,Technology,2862.675,5,0.1,763.2750,897.35,Critical
29781,30570,IN-2014-TS2134092-41949,6/11/2014,8/11/2014,First Class,TS-2134092,Toby Swindell,Consumer,NaN,Porirua,Wellington,New Zealand,Oceania,Asia Pacific,FUR-CH-5378,"Novimex Executive Leather Armchair, Adjustable",Chairs,Furniture,1822.080,4,0.0,564.8400,894.77,Critical
29722,31192,IN-2015-MB1808592-42108,14/4/2015,18/4/2015,Standard Class,MB-1808592,Mick Brown,Consumer,NaN,Hamilton,Waikato,New Zealand,Oceania,Asia Pacific,FUR-TA-3764,"Chromcraft Conference Table, Fully Assembled",Tables,Furniture,5244.840,6,0.0,996.4800,878.38,High
42059,40155,CA-2017-JW15220140-43022,14/10/2017,21/10/2017,Standard Class,JW-152201404,Jane Waco,Corporate,95823.0,Sacramento,California,United States,Western US,USCA,OFF-BI-4276,Fellowes PB500 Electric Punch Plastic Comb Bin...,Binders,Office Supplies,5083.960,5,0.2,1906.4850,867.69,Low
46942,40936,CA-2015-JH15985140-42031,27/1/2015,30/1/2015,Second Class,JH-159851408,Joseph Holt,Consumer,28027.0,Concord,North Carolina,United States,Southern US,USCA,FUR-TA-3753,Chromcraft Bull-Nose Wood Oval Conference Tabl...,Tables,Furniture,4297.644,13,0.4,-1862.3124,865.74,Critical


The products with very high shipping costs mostly have first or second class ship modes. They are also either fragile (e.g phones, digital accessories) or bulky (e.g furniture).

It is possible that customers paid a large premium to expedite high-quality shipping. This can be investigated further.

However, as these datapoints are plausible, I will not drop them.

### **Returns dataset**

In [39]:
# Check example rows of returns dataset
returns.head()

,Returned,Order ID,Region
0,Yes,IN-2017-CA120551-42816,Southern Asia
1,Yes,IN-2017-AA103751-42926,Southern Asia
2,Yes,IN-2017-TS212051-42904,Southern Asia
3,Yes,AG-2014-RO97803-41695,North Africa
4,Yes,AG-2015-LC70503-42265,North Africa


In [40]:
# Check for duplicated values
returns.duplicated(subset = ['Order ID']).sum()

np.int64(63)

In [48]:
# Drop duplicate rows
returns = returns.drop_duplicates()

# Export cleaned returns dataset
returns.to_csv('returns_clean.csv', index = False)

Subsequently, I will conduct further analysis and build the dashboard on Tableau.